# Tutorial: Creating Recommender Systems Datasets in Scientific Fields


<ul>
    <li><a href="#1">1. Data retrieval and cleaning</a></li>
</ul>
   
<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#1.1">1.1.Import libraries</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#1.2">1.2. Retrieve CORD-19</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#1.3">1.3. Exploring the articles of the dataset</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#1.4">1.4. Selecting a sample of articles to build our scientific recommendation dataset</a></li>
</ul>

<ul>
   <li><a href="#2">2. Named Entity Recognition (NER) + Named Entity Linking (NEL)</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#2.1">2.1. Import libraries</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#2.2">2.2. Configure MER</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#2.3">2.3. Import stop words vocabulary and tokenizer</a></li>
</ul>


<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#2.4">2.4. Extract the entities in a single file</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#2.5">2.5. Create entity files</a></li>
</ul>

<ul>
   <li><a href="#3">3. Creating the recommendation dataset</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.1">3.1. Import libraries</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.2">3.2. Get all articles id that cannot be considered in use case: blacklist </a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.3">3.3. Create the dataset like < user, item, rating, year> </a></li>
</ul>


<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.4">3.4. Get entities labels</a></li>
</ul>

<ul>
   <li>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#3.5">3.5. Save data</a></li>
</ul>

<ul>
   <li><a href="#4">4. Data statistics</a></li>
</ul>

















<a id="1"></a>

## 1. Data retrieval + cleaning

**Goal**: To retrieve the [COVID-19 Open Research Dataset (CORD-19)](https://www.semanticscholar.org/cord19) and to select a sample of complete English articles (authors' info, title, body text) to build a scientific recommendation dataset.

CORD-19 includes coronavirus-related research articles extracted from several sources, such as PubMed, bioRxiv, medRxiv, WHO.

<a id="1.1"></a>
### 1.1 Import libraries

In [1]:
import json
import os
import pandas as pd
import requests
import sys
import langdetect
from langdetect import detect
sys.path.append("./")

<a id="1.2"></a>
### 1.2. Retrieve CORD-19

CORD-19 is a large dataset, so in this tutorial we are going to use a smaller version of the dataset. 
This version is located under the directory "cord19_small":

In [32]:
os.chdir('data')
os.system('tar -xvf cord19_small.tar.xz')
os.chdir('../')

However, if you want to retrieve the entire dataset, you can run the following code:

In [ ]:
os.chdir('data')
         
version = 'cord-19_2020-05-12.tar.gz'

url = 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/' + version

response = requests.get(url)

os.system('tar -xvf cord-19_2020-05-12.tar.gz')
os.chdir('../')

Let's explore the contents of the dataset directory, particularly, the metadata file.

In [ ]:
dataset_dir = 'data/cord19_small/'
metadata_filepath = dataset_dir + 'metadata.csv'

metadata = pd.read_csv(metadata_filepath, sep = ',', quotechar = '"',  encoding = 'utf-8', dtype=str) 

Now we have a DataFrame with the contents of the metadata file.

To print column names and first row:

In [ ]:
metadata.head(1)

To access individual rows/articles:

In [ ]:
metadata.loc[0]

To access the individual column title':

In [ ]:
metadata['title']

Let's check the summary statistics:

In [ ]:
metadata.count()

By looking at the statistics, we can see that the number of records with data for the column 'cord_uid' is higher than the number of records with data for the column 'authors'. 

For our dataset, we only want to include articles with the following characteristics:
- authors' information
- available title
- available body text
- article text expressed in English
- non-duplicate articles

<a id="1.3"></a>

### 1.3. Exploring the articles of the dataset

Let's consider the first article appearing in the metadata file.
To check if information about the article's author is available:

In [ ]:
metadata.loc[0]['authors']

To check if there is an available title:

In [ ]:
metadata.loc[0]['title']

We can see that the title is expressed in English, but it would not be efficient to check the language of every article in the dataset, so we will apply the language detection tool [langdetect](https://pypi.org/project/langdetect/):

In [ ]:
title1 = metadata.loc[0]['title']

title1_lang = detect(title1)

print("Title:", title1, "\nLanguage:", title1_lang)

The tool detects English as the language of the title.

Let's check another article:

In [ ]:
title2 = metadata.loc[199]['title']

title2_lang = detect(title2)

print("Title:", title2, "\nLanguage:", title2_lang)

In this case, the tool detects german ('de') as the language of the title.

Now, we want to check if there is an available abstract for the first article:

In [ ]:
metadata.loc[0]['abstract']

The metadata file does not contain the article's text besides abstract and title, but we can access the file associated with the article using the provided information in the columns 'pdf_json_files' or 'pmc_json_files':

In [ ]:
metadata.loc[0]['pdf_json_files']

Let's open the file, which is in [JSON](https://www.json.org/json-en.html) format:

In [ ]:
article1_filepath = dataset_dir + metadata.loc[0]['pdf_json_files']

with open(article1_filepath, encoding='utf-8') as article1_file:
    article1_data = json.load(article1_file)

Now we have the file content stored in a dictionary with the following keys:

In [ ]:
article1_data.keys()

The article content is the following:

In [ ]:
article1_data

Let's check the body text:

In [ ]:
body_text = article1_data['body_text']
body_text

We have a list of dictionaries, each dictionary is a paragraph belonging to a given section of the article. We want to join the different parts text in a single string: 

In [ ]:
article1_text = str()

for paragraph in body_text:
    article1_text += paragraph['text'] + '\n'

article1_text

All good! We will include this article in our dataset.

<a id="1.4"></a>

### 1.4. Selecting a sample of articles to build our scientific recommendation dataset

Instead of repating each operation for each file individually, let us adapt our code to automatically select a sample containing 100 preprocessed articles. 

First, create the output directory:

In [33]:
out_dir = 'data/sample/'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

Then, initiallize the necessary variables:

In [34]:
max_articles = 100 #number of articles to include in the sample
dataset_dir = 'data/cord19_small/'
metadata_filepath = dataset_dir + 'metadata.csv'
valid_articles_count = int()
out_articles_ids = list()

Open the metadata file:

In [35]:
metadata = pd.read_csv(metadata_filepath, sep = ',', quotechar = '"',  encoding = 'utf-8', dtype=str) 

Then iterate over the records in the metadata file and choose only the relevant ones:

In [36]:
valid_articles_count = int()
blacklist = str()
blacklist_count = int()

for index, record in metadata.iterrows():
    invalid_article = True
    
    if valid_articles_count <= max_articles:
        
        if record['pubmed_id'] not in out_articles_ids:
            
            if type(record['sha']) != float:

                if record['authors'] != '':    

                    if record['title'] != '':
                        title = record['title']
                        title_lang = detect(title)
                        article_filepath = record['pdf_json_files']
                        
                        if article_filepath != '': # to consider onyl articles from the pdf_json directory

                            if title_lang == 'en'  \
                                and type(article_filepath) != float  \
                                and article_filepath.count("document") == 1:

                                article_filepath_up = dataset_dir + record['pdf_json_files']

                                with open(article_filepath_up, encoding='utf-8') as article_file:
                                    article_data = json.load(article_file)

                                if 'body_text' in article_data.keys(): # the article is valid
                                    valid_articles_count += 1
                                    invalid_article = False

                                    # open the article file to check if it contains all info
                                    with open(article_filepath_up) as article_file:
                                        article_data = json.load(article_file)

                                    # correct the info of the article with info present in metadata file
                                    changed_article = False

                                    if article_data['metadata']['title'] == '':
                                        article_data['metadata']['title'] = record['title']
                                        changed_article = True

                                    if article_data['metadata']['authors'] == []:
                                        authors = record['authors'].split(';')   
                                        
                                        add_author = {
                                                     'first': '', 'middle': [], 'last': '',
                                                     'suffix': '', 'affiliation': {}, 'email': ''
                                                     }
                                        authors_up = list()            
                                        
                                        for author in authors:
                                            add_author['last'] = author.split(',')[0]
                                            
                                            begin_names = author.split(',')[1].split(' ')
                                            add_author['first'] = begin_names[1]
                                            
                                            if len(begin_names) == 3:
                                                add_author['middle'] = [begin_names[2]]
                                            
                                            authors_up.append(add_author)
                                            
                                        article_data['metadata']['authors'] = authors_up
                                        changed_article = True
                                        
                                    # output or copy article file to out_dir
                                    if changed_article:
                                      
                                        with open(out_dir + record['sha'] + '.json', 'w') as out_file:
                                            out_file.write(json.dumps(article_data, indent=4, ensure_ascii=False))

                                    else:
                                        command = 'cp '  \
                                                  + article_filepath_up + ' ' \
                                                  + out_dir  \
                                                  + record['sha'] + '.json'

                                        os.system(command)
        
        if invalid_article: # store article pubmed id in blacklist file
            blacklist += record['pubmed_id'] + "\n"
            blacklist_count += 1
            
    if valid_articles_count == max_articles:
        total_articles = index + 1
        break

#Create blacklist file with info about invalid articles
with open('data/blacklist/blacklist_articles.txt', 'w') as blacklist_file:
    blacklist_file.write(blacklist)
    blacklist_file.close()

print("Invalid articles:", str(blacklist_count))

Invalid articles: 1


If you were not able to run the code, you can uncompress the file 'sample.tar.xz' under 'data' directory.

Let's check if the output directory contain the desired number of articles (max_articles):

In [37]:
article_count = len(os.listdir(out_dir))
assert article_count == max_articles, 'Invalid number of article(s): {}! Expected number: {}'.format(article_count, max_articles)

At the end of this section, we now have a sample including 100 articles that will be the basis of our scientific recommendation dataset.

<a id="2"></a>

## 2. Named Entity Recognition (NER) + Named Entity Linking (NEL)

**Goal**: To recognize chemical and disease entities in the retrieved articles and to link them to the respective ontology identifiers.

We are going to use the [Disease Ontology](https://disease-ontology.org/) (DO), and the [Chemical Entities of Biological Interest](https://www.ebi.ac.uk/chebi/) (ChEBI) ontology.

To perform NER and NEL, we are going to apply Minimal Named-Entity Recognizer [MER](https://pypi.org/project/merpy/) tool.

<a id="2.1"></a>
### 2.1. Import libraries

In [38]:
import merpy
import multiprocessing
from collections import Counter

<a id="2.2"></a>
### 2.2. Configure MER

First, we are going to download the ontologies files:

In [43]:
os.chdir('data/ontologies')

#Donwload DO (2021-06-03 version)
os.system('wget https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/v2021-06-03/src/ontology/releases/doid.owl')
merpy.download_lexicon('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/v2021-06-03/src/ontology/releases/doid.owl',
                       'doid', ltype='owl')

#Donwload ChEBI (2021-07-01 version)
os.system('wget ftp://ftp.ebi.ac.uk/pub/databases/chebi/archive/rel201/ontology/chebi_lite.owl.gz')
os.system ('gzip -d chebi_lite.owl.gz')
merpy.download_lexicon('ftp://ftp.ebi.ac.uk/pub/databases/chebi/archive/rel201/ontology/chebi_lite.owl',
                       'chebi', ltype='owl')

os.chdir('../../')

wrote doid lexicon
wrote chebi lexicon


Then, we need to process the downloaded files into lexicons that MER can use:

In [113]:
merpy.process_lexicon("chebi", ltype="owl")

merpy.process_lexicon("doid", ltype="owl")

z.val.ala.asp.ome..ch2f
z.val.phe.h
zwitterion
zygadenine
zymosterol
.z.z..2.9.16.heptadecatriene.4.6.diyn.8.ol
.z.z..alpha.farnesene
.z.z.z..geranylgeraniol
.z.z.z..icosa.8.11.14.trienoylethanolamide
.z.z.z..octadeca.4.7.10.trienenitrile
zwitterionic group
....zwittermicin a
....zwittermicin a
zwittermicin a
zxi 8901
zymonic acid
zymosterol ester
zymosteryl oleate
zymosteryl palmitoleate
.z.z..3.chloromuconic acid
zinc sulfate hexahydrate
ziprasidone hydrochloride hydrate
ziprasidone mesylate trihydrate
zirconium coordination entity
zirconium molecular entity
.z..methyl 3..methylsulfinyl..1.propenyl disulfide
zucchini factor b
zwitterionic polysialic acid acetylated at o.9
zymosterol intermediate 1c
zymosterol intermediate 2
zinc oxide
zinc sulfate
ziprasidone hydrochloride
ziprasidone mesylate
zirconium coordination
zirconium molecular
.z..methyl 3..methylsulfinyl..1.propenyl
zucchini factor
zwitterionic polysialic
zymosterol intermediate
 
xpid
xpv
xrn
xscid
yaba
yaws
zaspopathy
zoo

We are going to delete obsolete concepts still present in the ontologies file:

In [115]:
merpy.delete_obsolete("chebi")

merpy.delete_obsolete("doid")

We need to repeat the operations for the DO:

Let's check the lexicons available for MER:

In [116]:
merpy.show_lexicons()

lexicons preloaded:
['lexicon', 'doid', 'chebi']

lexicons loaded ready to use:
['chebi', 'doid']

lexicons with linked concepts:
['doid', 'lexicon', 'chebi']


<a id="2.3"></a>
### 2.3. Import stop words vocabulary and tokenizer

Stop words are common words of a given language (for example the words 'the', 'and', 'in'). A typical pre-processing step is to tokenize the text and remove the stopwords. For that, we are going to import NLTK's list of english stopwords and use the NLTK tokenizer.

In [39]:
import nltk 
nltk.download('punkt')

from nltk.corpus import stopwords

nltk.download('stopwords')

from nltk.tokenize import word_tokenize

all_stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /home/pruas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pruas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We are going to extend the stop words vocabulary by adding stop words associated with ChEBI and DO:

In [40]:
kbs_stopwords = list()
blacklist_dir = 'data/blacklist/'
filenames = ['chebi.txt', 'doid.txt']

for filename in filenames:
    
    with open(blacklist_dir + filename, 'r') as backlist_file:
        stopwords = [content.strip('\n') for content in backlist_file.readlines()]
        kbs_stopwords.extend(stopwords)
        backlist_file.close()

#Extend stop words vocabulary with the retrieved KBs stopwords
all_stopwords.extend(kbs_stopwords)

<a id="2.4"></a>
### 2.4. Extract the entities in a single file

Let's retrieve a file from the articles sample:

In [119]:
dataset_dir = 'data/sample/'

with open(dataset_dir + '87390d2ae28407b3e03e60a6b24a7fd99ed7229a.json') as article1_file:
    article_data = json.load(article1_file)

Let's check the contents of the article:

In [120]:
article_data.keys()

dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])

We want to recognize the entities present in title, abstract, and body. 

First, let's retrieve the title, which is a value associated with the key 'metadata':

In [121]:
title = article_data['metadata']['title']
title

'Pro/con clinical debate: Steroids are a key component in the treatment of SARS'

Let's tokenize the title:

In [122]:
title_tokens = word_tokenize(title)
title_tokens

['Pro/con',
 'clinical',
 'debate',
 ':',
 'Steroids',
 'are',
 'a',
 'key',
 'component',
 'in',
 'the',
 'treatment',
 'of',
 'SARS']

And remove the tokens relative to stop words:

In [123]:
title_tokens_up = [word for word in title_tokens if not word in all_stopwords]
title_tokens_up

['Pro/con',
 'clinical',
 'debate',
 ':',
 'Steroids',
 'key',
 'component',
 'treatment',
 'SARS']

And rebuild the title without the stop words:

In [124]:
title_up = (' ').join(title_tokens_up)
title_up

'Pro/con clinical debate : Steroids key component treatment SARS'

Then, we apply MER to the preprocessed title in order to recognize disease entities and to link them to DO concepts:

In [125]:
merpy.get_entities(title_up, 'doid')

[['59', '63', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945']]

Let's check if the annotations make sense. For instance, access the link http://purl.obolibrary.org/obo/DOID_2945.

The entity 'SARS' in the article was linked to the DO concept 'severe acute respiratory syndrome' with the identifier 'DOID:2945', which seems correct!

Let's apply MER to recognize chemical entities and to link them to ChEBI concepts:

In [126]:
merpy.get_entities(title_up, 'chebi')

[['']]

Accessing the link http://purl.obolibrary.org/obo/CHEBI_35341, we can see that the entity 'Steroids' was linked to the ChEBI concept 'steroid', which has the identifier 'CHEBI:35341'.

We add the disease and chemical entities to a single list:

In [127]:
title_entities = merpy.get_entities(title_up, 'doid') + merpy.get_entities(title_up, 'chebi')
title_entities_up = [entity for entity in title_entities if entity != ['']]

title_entities_up

[['59', '63', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945']]

Now, we are going to apply MER to recognize entities in abstract:

In [128]:
abstract = article_data['abstract'][0]['text']

#Tokenize and remove stop words
abstract_tokens = word_tokenize(abstract)
abstract_tokens_up = [word for word in abstract_tokens if not word in all_stopwords]
abstract_up = (' ').join(abstract_tokens_up)

#Entity recognition and linking 
abstract_entities =  merpy.get_entities(abstract_up, 'doid') + merpy.get_entities(abstract_up, 'chebi')
abstract_entities_up = [entity for entity in abstract_entities if entity != ['']]

abstract_entities_up

[['4', '8', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['87', '91', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['203', '207', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['318', '322', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['407', '411', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['479', '486', 'steroid', 'http://purl.obolibrary.org/obo/CHEBI_35341']]

Let's apply MER in the text associated with the body of the article:

In [129]:
body = str()

for section in article_data['body_text']:
    body += section['text'] + "\n"
    
#Tokenize and remove stop words
body_tokens = word_tokenize(body)
body_tokens_up = [word for word in body_tokens if not word in all_stopwords]
body_up = (' ').join(body_tokens_up)

#Entity recognition and linking 
body_entities =  merpy.get_entities(body_up, "doid") + merpy.get_entities(body_up, "chebi")
body_entities_up = [entity for entity in body_entities if entity != ['']]

body_entities_up

[['0', '4', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['51', '55', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['460', '464', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['558', '562', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['981', '985', 'SARS', 'http://purl.obolibrary.org/obo/DOID_2945'],
 ['1023', '1027', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1060', '1064', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1131', '1135', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1172', '1176', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1199', '1203', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1313', '1317', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1383', '1387', 'ARDS', 'http://purl.obolibrary.org/obo/DOID_11394'],
 ['1796', '1805', 'pneumonia', 'http://purl.obolibrary.org/obo/DOID_552'],
 ['1852', '1861', 'varicella', 'http://purl.obolibrary.org/obo/DOID_8659'],
 ['186

At last, we need to obtain information about the frequency of each ontology identifier in the document:

In [130]:
total_entities = title_entities_up + abstract_entities_up + body_entities_up

all_uris = [entity[3] for entity in total_entities]

entity_counter = Counter(all_uris)

entity_counter

Counter({'http://purl.obolibrary.org/obo/DOID_2945': 28,
         'http://purl.obolibrary.org/obo/DOID_11394': 15,
         'http://purl.obolibrary.org/obo/CHEBI_35341': 10,
         'http://purl.obolibrary.org/obo/DOID_552': 2,
         'http://purl.obolibrary.org/obo/DOID_8659': 1,
         'http://purl.obolibrary.org/obo/DOID_13564': 1,
         'http://purl.obolibrary.org/obo/DOID_423': 1,
         'http://purl.obolibrary.org/obo/DOID_1389': 1,
         'http://purl.obolibrary.org/obo/DOID_934': 1,
         'http://purl.obolibrary.org/obo/DOID_11162': 3,
         'http://purl.obolibrary.org/obo/DOID_10533': 1,
         'http://purl.obolibrary.org/obo/CHEBI_50858': 1,
         'http://purl.obolibrary.org/obo/CHEBI_50906': 3})

To sort the URIs by descending order:

In [131]:
entity_counter = {
    k: v 
    for k, v in sorted(entity_counter.items(), key=lambda item: item[1], reverse=True)
    }

entity_counter 

{'http://purl.obolibrary.org/obo/DOID_2945': 28,
 'http://purl.obolibrary.org/obo/DOID_11394': 15,
 'http://purl.obolibrary.org/obo/CHEBI_35341': 10,
 'http://purl.obolibrary.org/obo/DOID_11162': 3,
 'http://purl.obolibrary.org/obo/CHEBI_50906': 3,
 'http://purl.obolibrary.org/obo/DOID_552': 2,
 'http://purl.obolibrary.org/obo/DOID_8659': 1,
 'http://purl.obolibrary.org/obo/DOID_13564': 1,
 'http://purl.obolibrary.org/obo/DOID_423': 1,
 'http://purl.obolibrary.org/obo/DOID_1389': 1,
 'http://purl.obolibrary.org/obo/DOID_934': 1,
 'http://purl.obolibrary.org/obo/DOID_10533': 1,
 'http://purl.obolibrary.org/obo/CHEBI_50858': 1}

<a id="2.5"></a>
### 2.5. Create entity files

We need to adapt our code to perform NER and NEL in all documents of our sample.

First, create the output dir:

In [41]:
out_dir = 'data/sample_entities/'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

Next, we are going to iterate over each file present in the sample directory, annotate it, and create the respective entity file:

In [42]:
def annotate_doc(article):
    
    article_filepath = 'data/sample/' + article  

    with open(article_filepath) as input_file:
        article_data = json.load(input_file)

    doc_output = {'id': str(), 'entities': {}, 'sections': {'title': [], 'abstract': [], 'body': []}}
    
    doc_output['id'] = article_data['paper_id']
   
    #Annotate the title
    title = article_data['metadata']['title']
    
    #Tokenize and remove stop words
    title_tokens = word_tokenize(title)
    title_tokens_up = [word for word in title_tokens if not word in all_stopwords]
    title_up = (' ').join(title_tokens_up)
    
    #Entity recognition and linking
    title_entities = merpy.get_entities(title_up, 'doid') + merpy.get_entities(title_up, 'chebi')
    doc_output['sections']['title'] = [entity for entity in title_entities if entity != ['']]
    
    #Annotate the abstract
    if article_data['abstract'] != []:
        abstract = article_data['abstract'][0]['text']
        
        #Tokenize and remove stop words
        abstract_tokens = word_tokenize(abstract)
        abstract_tokens_up = [word for word in abstract_tokens if not word in all_stopwords]
        abstract_up = (' ').join(abstract_tokens_up)
        
        #Entity recognition and linking
        abstract_entities = merpy.get_entities(abstract_up, 'doid') + merpy.get_entities(abstract_up, 'chebi')
        doc_output['sections']['abstract'] = [entity for entity in abstract_entities if entity != ['']]
    
    else:
        abstract_entities = []

    #Combine the several paragraphs of the body text and annotate it
    body = str()

    for section in article_data['body_text']:
        body += section['text'] + '\n'
    
    #Tokenize and remove stop words
    body_tokens = word_tokenize(body)
    body_tokens_up = [word for word in body_tokens if not word in all_stopwords]
    body_up = (' ').join(body_tokens_up)    
        
    #Entity recognition and linking
    body_entities = merpy.get_entities(body_up, 'doid') + merpy.get_entities(body_up, 'chebi')
    doc_output['sections']['body'] = [entity for entity in body_entities if entity != ['']]

    # Count URIs frequencies and sort them
    total_entities = title_entities + abstract_entities + body_entities
    all_uris = [entity[3] for entity in total_entities if len(entity)==4]
    entity_counter = Counter(all_uris)
    
    doc_output['entities'] = {
        k: v 
        for k, v in sorted(entity_counter.items(), key=lambda item: item[1], reverse=True)
        }
    
    #Generate JSON file with output
    out_filepath = out_dir + doc_output['id'] + '_entities.json'
    
    with open(out_filepath, 'w') as out_file:
        out_file.write(json.dumps(doc_output, indent=4))
        

article_dir = 'data/sample/'
        
with multiprocessing.Pool(processes=10) as pool: 
    # change the number of processes according to number of available cores
    outputs = pool.map(annotate_doc, [article for article in os.listdir(article_dir)], chunksize=10)
    pool.close()

If you were not able to run the previous code, you can extract the file 'sample_entities.tar.xz' under 'data' directory

Now we have both the article files ('data/sample' directory) and the respective entities files ('data/sample_entities' directory), and the next step will be the generation of the scientific recomendation dataset using the LIBRETTI algorithm.

<a id="3"></a>
## 3. Create the recommendation dataset

<a id="3.1"></a>
### 3.1. Import libraries

In [8]:
import numpy as np
import time
from datetime import date, datetime

from pathlib import Path
import rdflib
from rdflib import URIRef

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

Path of original' json and entities' json folder, blacklist and metadata

In [29]:
dataset_dir = 'data/sample/'
entities_dir = 'data/sample_entities/'
metadata_dir = 'data/metadata.csv'
blacklist_dir = 'data/blacklist/blacklist_articles.txt'

List containing the names of the entries in the directory given by path

In [24]:
entities_list_of_json_files = os.listdir(entities_dir)
print(entities_list_of_json_files)

['23bc55d6f63fab18b02004483888db2b6a0bfa48_entities.json', '3bda17d21aee670c29e22635d622b780820572af_entities.json', '65cc7496f21429d81b3ae129d9c39764e2a1f568_entities.json', 'dce8be353f8ed76957af86118e1b77fd82f54292_entities.json', 'c63c4d58d170136b8d3b5a66424b5ac3f73a92d9_entities.json', '06ced00a5fc04215949aa72528f2eeaae1d58927_entities.json', '3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c_entities.json', '867e1b0f6ca8757f2a32a625d99b23888ab40d49_entities.json', '467694c7a219031c9be1734c7ab3bc42bfa07590_entities.json', '26bdc177821e0161d138236656dafefe2f9f6900_entities.json', 'dab2d95da5307c840eb4a1e00f0186022541d63d_entities.json', '0104f6ceccf92ae8567a0102f89cbb976969a774_entities.json', 'e3af2ca43010f59c3d1bb731abd011e3dd0fc51c_entities.json', 'c6008b68c8b16e3a6a48a2cb892bac5c9353df86_entities.json', '017ca5bdac589a37196df7b8e077c4c371ab32da_entities.json', 'def1cf77e1ef84f4373a342e23145be05ec5e226_entities.json', '52ce19fe79de46d9bb0fa7e75a0631fed2d3efd2_entities.json', 'd77233dd9085

<a id="3.2"></a>
### 3.2. Get all articles id that cannot be considered in use case: blacklist 

In [27]:
# Return all articles to be removed, due some errors found there

articles_blacklist = []
with open(blacklist_dir, 'r') as f:
    black_list = [content for content in f.readlines()]

articles_blacklist.extend(black_list) 
f.close()

print(articles_blacklist)

[]


In [30]:
metadata = pd.read_csv(metadata_dir)

<a id="3.3"></a>
### 3.3. Create the dataset like <user, item, rating, year> 

In [31]:
user_item_rating_all = []
count = 0

#entities_list_of_json_files=['PMC7176216_entities.json']

for file in entities_list_of_json_files:
    
    if file.replace('_entities.json','') in articles_blacklist:
        continue
     
    print(count, "-", len(entities_list_of_json_files))            
    print(file)
        
    # check valid json file, i.e. contains values
    try:
        j_file_entities = open_json_file_pd(path=entities_dir, file=file)           
    except Exception as e:
        print(f'Json file does not contain values. Error message {e}')
        with open(blacklist_dir, 'r+') as blacklist_file:
            content = blacklist_file.read()
            blacklist_file.seek(0, 0)
            blacklist_file.write(file.replace('_entities.json','').rstrip('\r\n') + '\n')
            blacklist_file.write(content)
            blacklist_file.close()
        continue  

    my_dict = j_file_entities.loc['entities'][0]  
    df_entities = pd.DataFrame(my_dict.items(), columns=['entities', 'count'])
    
    df_entities['entities_id'] = df_entities.entities.str.split(pat="/").str[-1]
   
    if df_entities.empty:
        print(f'Json file does not contain values.')
        with open(blacklist_dir, 'r+') as blacklist_file:
            content = blacklist_file.read()
            blacklist_file.seek(0, 0)
            blacklist_file.write(file.replace('_entities.json','').rstrip('\r\n') + '\n')
            blacklist_file.write(content)
            blacklist_file.close()
        continue     
    
    #print(df_entities)
    
    article_id = j_file_entities.loc['id'].values[0]
    
    # check valid json file, i.e. contains values
    try:
        # Convert a JSON string to pandas object, and return a dataframe
        with open(dataset_dir + article_id + '.json', encoding='utf-8') as json_file:
            j_file_original = json.load(json_file)
        #print(j_file_original)
    except Exception as e:
        print(f'Original json file does not exist. Error message {e}')
        #This function receives the article to and saves them in the
        # blacklist file. The backlist contains all invalid articles, such
        # as non-authors, non-entities, and others
        with open(blacklist_dir, 'r+') as blacklist_file:
            content = blacklist_file.read()
            blacklist_file.seek(0, 0)
            blacklist_file.write(file.replace('_entities.json','').rstrip('\r\n') + '\n')
            blacklist_file.write(content)
            blacklist_file.close()
        continue    
    
    # check if json file contains AUTHORS, otherwise try to find them in metadata.csv
    # if value remains null them put this article in the blacklist file
    
    list_of_authors = []
    for p in j_file_original['metadata']['authors']:
        
        if len(p['first']) == 0 or len(p['last']) == 0:
            continue
        else:
            # remove all characters except alphabets from a string to unidecode
            first = unidecode.unidecode( ''.join(m for m in p['first'] if m.isalpha()))
            middle = unidecode.unidecode( ''.join(m for m in p['middle'] if m.isalpha()))
            last = unidecode.unidecode( ''.join(m for m in p['last'] if m.isalpha()))

            list_of_authors.append(first + ', '+ middle + ' '+ last)
    
###  ---- Begin of Metadata file ----
    
    # if authors is empty we will find in metadata.csv file
    if len(list_of_authors)==0:
        ##if string is NaN
        try:
            authors = data[metadata.sha == article_id].authors.values[0].split(';')
            for a in authors:
                a = a.split(',')
                first = unidecode.unidecode( ''.join(m for m in a[1] if m.isalpha()))
                last = unidecode.unidecode( ''.join(m for m in a[0] if m.isalpha()))
                list_of_authors.append(first + ', '+  last) 
        
        except Exception as e:
            print(f'Empty values {e}')   
            #This function receives the article to and saves them in the
            # blacklist file. The backlist contains all invalid articles, such
            # as non-authors, non-entities, and others
            with open(blacklist_dir, 'r+') as blacklist_file:
                content = blacklist_file.read()
                blacklist_file.seek(0, 0)
                blacklist_file.write(file.replace('_entities.json','').rstrip('\r\n') + '\n')
                blacklist_file.write(content)
                blacklist_file.close()               
        continue
        
    # CHECK YEAR
    
    publish_date=''      
    try:
        publish_date = metadata[metadata.sha == article_id].\
            publish_time.map(lambda v: v.split('-')[0]).tolist()[0]
    finally:
        pass  
    print(publish_date)
    
    if publish_date==None:
        #This function receives the article to and saves them in the
        # blacklist file. The backlist contains all invalid articles, such
        # as non-authors, non-entities, and others
        with open(blacklist_dir, 'r+') as blacklist_file:
            content = blacklist_file.read()
            blacklist_file.seek(0, 0)
            blacklist_file.write(file.replace('_entities.json','').rstrip('\r\n') + '\n')
            blacklist_file.write(content)
            blacklist_file.close()
        continue

    count+=1
    
###  ---- End of Metadata file ----            
    
    # Append <user, item, rating> tuple
    user_item_rating = []
    for author in list_of_authors:
        for entity in df_entities.entities_id:
            user_item_rating.append([author, entity, 1])

    #print(user_item_rating)
    
    ## add publish_date in array in index column = 3
    user_item_rating = np.insert(user_item_rating, 3, publish_date, axis=1)
    # print(user_item_rating)
    
    user_item_rating_all.append(user_item_rating)

0 - 100
23bc55d6f63fab18b02004483888db2b6a0bfa48_entities.json
Json file does not contain values. Error message name 'open_json_file_pd' is not defined
0 - 100
3bda17d21aee670c29e22635d622b780820572af_entities.json
Json file does not contain values. Error message name 'open_json_file_pd' is not defined
0 - 100
65cc7496f21429d81b3ae129d9c39764e2a1f568_entities.json
Json file does not contain values. Error message name 'open_json_file_pd' is not defined
0 - 100
dce8be353f8ed76957af86118e1b77fd82f54292_entities.json
Json file does not contain values. Error message name 'open_json_file_pd' is not defined
0 - 100
c63c4d58d170136b8d3b5a66424b5ac3f73a92d9_entities.json
Json file does not contain values. Error message name 'open_json_file_pd' is not defined
0 - 100
06ced00a5fc04215949aa72528f2eeaae1d58927_entities.json
Json file does not contain values. Error message name 'open_json_file_pd' is not defined
0 - 100
3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c_entities.json
Json file does not contai

In [19]:
flat_list = []
for sublist in user_item_rating_all:
    for item in sublist:
        flat_list.append(item)

In [20]:
final_data = pd.DataFrame(np.array(flat_list),  columns=['user', 'item', 'rating', 'year'])
sum_df = final_data.groupby(['user', 'item', 'year']).size().reset_index().rename(columns={0: 'rating'})   

#     maps the values to the lowest consecutive values
df_user_index = pd.DataFrame(sum_df.user.unique(), columns=["user"])
df_user_index["new_index"] = np.arange(0, len(sum_df.user.unique()))

sum_df["index_user"] = sum_df["user"].map(df_user_index.set_index('user')["new_index"]).fillna(0) 
df_with_user_id = sum_df

ValueError: Empty data passed with indices specified.

In [21]:
df_with_user_id.rename(columns={'user': 'author_name', 'index_user': 'user'}, inplace = True)

NameError: name 'df_with_user_id' is not defined

<a id="3.4"></a>
### 3.4. Get entities labels

In [ ]:
list_of_entities = df_with_user_id.item.unique()
print(list_of_entities) 

Loading ontologies: Get entities lables from http://purl.obolibrary.org/obo/ based on items prefix

In [ ]:
path_chebi = 'data/ontologies/chebi.owl'
path_do = 'data/ontologies/doid.owl'

In [ ]:
#     Get entities lables from http://purl.obolibrary.org/obo/ based on items prefix

entities_label = []
for id in list_of_entities: 
    uri = URIRef('http://purl.obolibrary.org/obo/' + id)
    if id.startswith('CHEBI'):
        label = chebi.label(uri)
    elif id.startswith('DO'):
        label = do.label(uri)
    entities_label.append(label)

print(entities_label)

In [ ]:
df_entities = pd.DataFrame(list_of_entities, columns=["item_id"])
df_entities["entity_name"] = np.array(entities_label)
print(df_entities)

Mapping labels, and get dataset as <user, author_name, item, rating, item_name, year>

In [ ]:
df_with_user_id["item_name"] = df_with_user_id["item"].map(df_entities.set_index('item_id')["entity_name"]).fillna(0)
print(df_with_user_id)

<a id="3.5"></a>
### 3.5. Save data

First, create the output dir 

In [ ]:
cord_ds_dir = 'data/results/sample_cord-19_dataset.csv'
cord_userid_dir = 'data/results/sample_cord-19_dataset_userid.csv'

if not os.path.exists(os.path.dirname(cord_ds_dir)):
        Path(os.path.dirname(cord_ds_dir)).mkdir(parents=True, exist_ok=True)   
        
if not os.path.exists(os.path.dirname(cord_userid_dir)):
        Path(os.path.dirname(cord_userid_dir)).mkdir(parents=True, exist_ok=True) 

Now, save all values in csv file        

In [ ]:
df_with_user_id[['user', 'item', 'rating', 'item_name', 'year']].to_csv(cord_ds_dir, index=False, header=False)

df_with_user_id[['user', 'author_name']].to_csv(cord_userid_dir, index=False, header=False) 

<a id="4"></a>
## 4. Data statistics 

In [ ]:
import pandas as pd

dataset = pd.read_csv('data/results/sample_cord-19_dataset.csv', names=['user', 'item', 'rating', 'item_name', 'year'])
#dataset = pd.read_csv('/home/mbarros/Desktop/data/scirec2021/comm_subset_cord-19_dataset_total_ordered_filtered.csv', names=['user', 'item', 'rating', 'item_name', 'year'])


print(dataset.head(20))

In [ ]:
# number of unique users
# number of unique items
# number of ratings
# sparsity 

print('n users: ', dataset.user.unique().shape[0])
print('n items: ', dataset.item.unique().shape[0])
print('n ratings: ', dataset.size)

print('sparsity: ', 1 - (dataset.size / (dataset.user.unique().shape[0] * dataset.item.unique().shape[0])))

In [ ]:
# items by ontology

print("CHEBI items: ", dataset[dataset.item.str.startswith('CHEBI')].item.unique().shape[0])

print("DOID items: ", dataset[dataset.item.str.startswith('DOID')].item.unique().shape[0])

In [ ]:
# items by user
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

unique_users = dataset.user.unique()
items_by_user = dataset.groupby(['user'])["item"].count().reset_index()
items_by_user = items_by_user.sort_values(by=['item'], ascending=False)
print(items_by_user)
items_by_user.user = items_by_user.user.astype('str')

print('max items by user: ', items_by_user.item.max())
print('min items by user: ', items_by_user.item.min())
print('mean items by user: ', items_by_user.item.mean())

In [ ]:
plt.scatter(items_by_user.user, items_by_user.item)
plt.axhline(y=items_by_user.item.mean(), color='r', linestyle='-')
plt.ylabel('number of items')
plt.xlabel('user')
plt.show()

In [ ]:
# users by item

unique_items = dataset.item.unique()
users_by_item = dataset.groupby(['item'])["user"].count().reset_index()
users_by_item = users_by_item.sort_values(by=['user'], ascending=False)

print('max users by item: ', users_by_item.user.max())
list_of_max_items = users_by_item[users_by_item.user == users_by_item.user.max()].item.values
print(list_of_max_items)

print(dataset[dataset.item == list_of_max_items[0]])

print('min users by item: ', users_by_item.user.min())
print('mean users by item: ', users_by_item.user.mean())

In [ ]:
dataset_doid = dataset[dataset.item.str.startswith('DOID')]
unique_items_d = dataset_doid.item.unique()
users_by_item_d = dataset_doid.groupby(['item'])["user"].count().reset_index()
users_by_item_d = users_by_item_d.sort_values(by=['user'], ascending=False)

print('max users by item: ', users_by_item_d.user.max())
list_of_max_items_d = users_by_item_d[users_by_item_d.user == users_by_item_d.user.max()].item.values
print(list_of_max_items_d)

print(dataset_doid[dataset_doid.item == list_of_max_items_d[0]])

print('min users by item: ', users_by_item_d.user.min())
print('mean users by item: ', users_by_item_d.user.mean())

In [ ]:
dataset_chebi = dataset[dataset.item.str.startswith('CHEBI')]
unique_items_c = dataset_chebi.item.unique()
users_by_item_c = dataset_chebi.groupby(['item'])["user"].count().reset_index()
users_by_item_c = users_by_item_c.sort_values(by=['user'], ascending=False)

print('max users by item: ', users_by_item_c.user.max())
list_of_max_items_c = users_by_item_c[users_by_item_c.user == users_by_item_c.user.max()].item.values
print(list_of_max_items_c)

print(dataset_chebi[dataset_chebi.item == list_of_max_items_c[0]])

print('min users by item: ', users_by_item_c.user.min())
print('mean users by item: ', users_by_item_c.user.mean())

In [ ]:
#%matplotlib qt

plt.scatter(users_by_item.item, users_by_item.user)
plt.axhline(y=users_by_item.user.mean(), color='r', linestyle='-')
#plt.yscale('log')
plt.ylabel('number of users')
plt.xlabel('items')
plt.show()